In [1]:
import PIL
from PIL import Image
import matplotlib

import numpy as np
import pandas as pd

import ast
import re

import json

### **Load Images**

Currently I saved all the images and their information in a csv, which means I don't need to open lots of images when I run my code, but it also means I have to decode the strings in the csv to get my images and labels

In [2]:
train_df = pd.read_csv("C:\\Users\\KurtJi\\OneDrive - University of Illinois - Urbana\\Desktop\\Personal Projects\\data\\inaturalist\\data_csv\\train_dataset.csv")
val_df = pd.read_csv("C:\\Users\\KurtJi\\OneDrive - University of Illinois - Urbana\\Desktop\\Personal Projects\\data\\inaturalist\\data_csv\\val_dataset.csv")
test_df = pd.read_csv("C:\\Users\\KurtJi\\OneDrive - University of Illinois - Urbana\\Desktop\\Personal Projects\\data\\inaturalist\\data_csv\\test_dataset.csv")

In [3]:
# get all unique category ids in the training data
train_cateid = np.unique(list(train_df['category_id']))

In [4]:
train_df.head()

,image_path,image_info,category_id
0,C:\Users\KurtJi\OneDrive - University of Illin...,"['2018-04-21 22:01:00+00:00', [27.83488, -97.0...",3198
1,C:\Users\KurtJi\OneDrive - University of Illin...,"['2019-07-27 22:30:48+00:00', [31.90883, -109....",9217
2,C:\Users\KurtJi\OneDrive - University of Illin...,"['2018-05-13 14:39:46+00:00', [-34.6054, -58.3...",1978
3,C:\Users\KurtJi\OneDrive - University of Illin...,"['2019-09-23 18:00:59+00:00', [38.73597, -90.7...",5244
4,C:\Users\KurtJi\OneDrive - University of Illin...,"['2018-06-08 18:01:00+00:00', [16.73704, -92.6...",4214


In [5]:
def get_image_label(cateid: int, categories, type: str):
    # categories should look like this: test_ann['categories']
    # print(categories)
    cate_list = list(categories[0][0].keys())[1:]
    # print(cate_list)
    if type not in cate_list:
        return "please enter correct category level"

    return categories[0][cateid][type]

In [6]:
# Define a function to load and preprocess images
def load_and_preprocess_images(df, type:str, catetype: str):
    image_data = []
    image_info = []
    labels = []

    if type not in ['train', 'val', 'test']:
        print("please enter correct data name!")
        return

    # Validating data
    with open(f"C:\\Users\\KurtJi\\OneDrive - University of Illinois - Urbana\\Desktop\\Personal Projects\\data\\inaturalist\\train_mini_sample.json", 'r') as file:
        # Load JSON data from the file
        ann = json.load(file)

    for index, row in df.iterrows():

        image_path = row['image_path']
        info = ast.literal_eval(row['image_info'])
        # get category name
        cate_id = row['category_id']

        # do not read the classes not in training data
        if type == 'val' or type == 'test':
            if cate_id not in train_cateid:
                # skip if category is not in training set
                continue
            
        # get label
        categories = ann['categories']
        label = get_image_label(cate_id, categories, catetype)
        # Load and preprocess the image
        image = Image.open(image_path)
        # reshape image
        image = image.resize((256, 256))
        # Apply any necessary preprocessing (resize, normalize, etc.)
        image = np.array(image)  # Convert to NumPy array if needed

        image_data.append(image)
        labels.append(label)

    return np.array(image_data), np.array(image_info), np.array(labels)

In [7]:
train_data, train_info, train_labels = load_and_preprocess_images(train_df, 'train', 'phylum')
val_data, val_info, val_labels = load_and_preprocess_images(val_df, 'val', 'phylum')
test_data, test_info, test_labels = load_and_preprocess_images(test_df, 'test', 'phylum')

In [8]:
np.shape(train_data[99])

(256, 256, 3)

In [9]:
len(train_data)

39997

## **Image Processing**

### **Denoising using fft**

## **Modeling**

#### **Encode class labels**

In [10]:
# get number of classes in training set
num_classes = len(np.unique(train_labels))

# get size of image
height, width, channels = 256, 256, 3

In [11]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(np.unique(train_labels))

LabelEncoder()

In [12]:
train_labels_int = le.transform(train_labels)
val_labels_int = le.transform(val_labels)
test_labels_int = le.transform(test_labels)

In [13]:
len(np.unique(train_labels_int))

13

#### **Check device**

In [14]:
!conda info --envs

# conda environments:
#
                         C:\Users\KurtJi\AppData\Local\R-MINI~1
                         C:\Users\KurtJi\AppData\Local\R-MINI~1\envs\r-reticulate
                         C:\Users\KurtJi\AppData\Local\r-miniconda
                         C:\Users\KurtJi\anaconda3
                         C:\Users\KurtJi\miniconda3
base                  *  c:\Users\KurtJi\anaconda3



**CUDA+PyTorch Tutorial** https://blog.csdn.net/qq_45956730/article/details/126600028

In [15]:
import torch
# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
torch.cuda.is_available()

True

In [17]:
device = torch.cuda.current_device()
torch.cuda.get_device_name(device)

'NVIDIA GeForce RTX 3070 Laptop GPU'

#### **Construct Model**

In [39]:
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [ ]:
## Parameters for model

# Hyper parameters
num_epochs = 8
num_classes = 2
batch_size = 128
learning_rate = 0.002

In [ ]:
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

dataset_train = MyDataset(df_data=train, data_dir=train_path, transform=trans_train)
dataset_valid = MyDataset(df_data=val, data_dir=train_path, transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)